In [7]:
# ensemble learning

In [5]:
def data_normalization(data):
    # 获取所有数值型数据 注意 这里目前传递的是引用，非其它数据类型
    numeric_features = data.dtypes[data.dtypes != 'object'].index

    data[numeric_features] = data[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
    # 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
    data[numeric_features] = data[numeric_features].fillna(0)

    return data

In [10]:
import numpy as np
import pandas as pd

csv_path = './PRSA_data_dropna.csv'

dt = pd.read_csv(csv_path)

X_pd = dt.loc[:,['DEWP','TEMP','PRES','cbwd','Iws','Is','Ir']] # 'year','month','day','hour',
Y_pd = dt.loc[:,'pm2.5']

# normalization
X_norm_pd = data_normalization(X_pd)

X = X_norm_pd.values.astype(float)
Y = Y_pd.values.astype(int)

Y = np.reshape(Y,(-1,1))

# TODO:这里按照project要求进行抽取 已完成
X_test, y_test = X[[i for i in range(X.shape[0]) if i % 7 == 6]][:, 0:], Y[[i for i in range(Y.shape[0]) if i % 7 == 6]][:, 0]
X_train, y_train = X[[i for i in range(X.shape[0]) if i % 7 != 6]][:, 0:], Y[[i for i in range(Y.shape[0]) if i % 7 != 6]][:, 0]


In [11]:
from sklearn.preprocessing import StandardScaler

# ss_X=StandardScaler()
# ss_y=StandardScaler()
#
# #标准化处理
# X_train=ss_X.fit_transform(X_train)
# X_test=ss_X.fit_transform(X_test)
# y_train=ss_y.fit_transform(y_train)
# y_test=ss_y.fit_transform(y_test)

#分别导入普通随机森林回归模型、极端随机森林回归模型、梯度提升树回归模型
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor

rfr=RandomForestRegressor()
rfr.fit(X_train,y_train)
rfr_y_predict=rfr.predict(X_test)


etr=ExtraTreesRegressor()
etr.fit(X_train,y_train)
etr_y_predict=etr.predict(X_test)

gbr=GradientBoostingRegressor()
gbr.fit(X_train,y_train)
gbr_y_predict=gbr.predict(X_test)


#对三种集成模型进行性能评价
print('R-squared value of RandomForestRegressor:',rfr.score(X_test,y_test))
print('R-squared value of ExtraTreesRegressor:',etr.score(X_test,y_test))
print('R-squared value of GradientBoostingRegressor:',gbr.score(X_test,y_test))

R-squared value of RandomForestRegressor: 0.4387778096511499
R-squared value of ExtraTreesRegressor: 0.40381754132988323
R-squared value of GradientBoostingRegressor: 0.3920382457288726


In [16]:
# model evaluation
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

lr_1 = rfr.fit(X_train, y_train)
# lr_1.predict(X_test)
r2_score(y_test, lr_1.predict(X_test)), r2_score(y_train, lr_1.predict(X_train))

(0.4445183600305871, 0.9030973391580317)

In [24]:
# knn 回归
from sklearn.neighbors import KNeighborsRegressor
reg = KNeighborsRegressor(n_neighbors=6)
reg.fit(X_train, y_train)

"""打印对于测试集的预测值"""
print("Test set predictions:\n{}".format(reg.predict(X_test)))

"""打印测试集上的误差"""
print("Test set R^2: {:.2f}".format(reg.score(X_test, y_test)))

Test set predictions:
[197.66666667 100.5         95.83333333 ...  13.5          9.
  11.33333333]
Test set R^2: 0.44


In [ ]:
# SVR regressor


.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

ValueError: Expected 2D array, got 1D array instead:
array=[33.8 20.3 10.2 22.  21.2 24.2 29.  22.7 21.8 34.9 25.2 20.9 19.4 20.
 14.  30.1 33.1 20.6 22.6 33.4 20.1 10.5 15.6 16.8 22.6 34.6 19.8 17.8
 22.  17.4 15.4 16.7 22.6 15.1 21.4 15.3  7.4 13.9 17.6 25.  46.7 17.1
 23.1 18.7 21.9 18.9 26.7 22.3 25.  14.6 42.8 17.3 22.2 36.5 22.8 19.9
 36.2 50.  25.  22.2 17.5 23.9 19.6 24.7 28.4  8.7 21.7 20.  19.9 24.5
 15.   7.  15.2 20.4  8.5 17.1 30.1 15.  19.4 23.2 17.  18.9 50.  25.
 46.   7.2 17.8 35.1 24.3  5.  16.6 21.8 28.5 22.  20.3 21.7 26.4 30.7
 50.  17.2 26.6 21.  23.4 19.5 20.7 23.3 48.8 15.6 19.6 17.4 21.7 14.6
 37.9  9.7 17.8 12.1 20.1 29.9 26.4 18.8 32.5 15.7 13.4 21.7 23.6 11.9
 13.8 22.2 13.  33.2 50.  22.3 22.4 23.8 29.1 20.8 23.7 19.8 13.9 28.4
 45.4 23.7 50.  18.  17.1 18.9 10.4 24.7 23.9 23.  20.2  8.5 14.2 20.3
 18.5 12.  19.3 20.6 16.1 12.3 23.1 22.7 20.3 16.7 27.9 21.4  8.1 37.6
 15.6 29.6 22.9 24.8 24.4 50.  28.7 50.  16.5 18.2 50.  16.2 14.1 21.2
 18.4 25.  50.  21.2